In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------


Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 10

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v08"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v08 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V10/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V10/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2019M04D04 UTC 13:35


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10/': No such file or directory


In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [8]:
df.shape

(16385, 1)

In [9]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [10]:
if TESTING:
    df = df[0:10]

In [11]:
df_split = np.array_split(df, cpu_count*100)

In [12]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [13]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//131902_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//127443_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115801_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//223940_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//142625_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//132895_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//155562_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//232144_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//162203_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//157807_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//181976_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//122784_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//153021_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//145102_V10.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//181993_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//172524_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//224002_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//132930_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//162209_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//232160_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//155571_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//132110_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//127461_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//122810_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//158116_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//153028_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115809_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//145109_V10.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//232240_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//132180_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//224009_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//127468_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//155579_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//116006_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//142642_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111045_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//158143_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//122845_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//184000_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//172542_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//132948_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//145207_V10.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//122853_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//225117_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//132955_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//127485_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111070_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//142649_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//155617_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//132225_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//162450_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//153042_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//145404_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//232303_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//158211_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//211017_V10.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//142657_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//132242_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//132972_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//162467_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//232401_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//211034_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//155635_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//122870_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//153049_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//158219_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111088_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//172558_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//145601_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//127494_V10.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//822492_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//742446_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//753308_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//914803_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//712700_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//742992_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//772350_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863096_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//831953_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//662903_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//723076_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//812109_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//652209_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//782704_V10.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//822499_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//713222_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//742481_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//723112_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//753406_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//812207_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//831960_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//642481_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//782802_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911106_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//726072_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//652306_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//743001_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//772404_V10.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911217_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//713230_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//723120_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//652405_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//726081_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//772601_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//812306_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//822519_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//664028_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//832105_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//742489_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911218_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//751201_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//652406_V10.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911265_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//751209_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//822546_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//832423_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//812460_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//652504_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//742497_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//772700_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//753622_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911266_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//726090_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//822547_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//751300_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//782980_V10.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//832447_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//812502_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//772940_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//822625_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911402_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//753680_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//783103_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//812503_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911403_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//832448_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//822626_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//772950_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//753690_V10.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//783104_V10.csv
/volumes/data/Y2018M

In [ ]:
#!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [ ]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111011_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111012_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111013_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111014_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111015_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111016_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111017_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111018_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//111019_V10.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//112130_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//112120_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//112140_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//112160_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//112150_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//112190_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//112180_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//112200_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//112300_V10.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//114445_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//114446_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//114447_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//114449_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//114460_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//114450_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//114448_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//114470_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//114480_V10.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115428_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115429_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115440_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115451_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115430_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115452_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115453_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115454_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//115455_V10.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//117201_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//117202_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//117203_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//117204_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//117205_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//117206_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//117207_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//117208_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//117209_V10.csv [Content-Type=text/

...

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863050_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863060_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863070_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863080_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863091_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863092_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863093_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863094_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//863095_V10.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911601_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911602_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911603_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911604_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911605_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911606_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911607_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911608_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//911609_V10.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//912600_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//912701_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//912702_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//912703_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//912704_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//912705_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//912706_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//912707_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//912708_V10.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//913605_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//913606_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//913607_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//913608_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//913609_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//913700_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//913801_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//913802_V10.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V10//913803_V10.csv [Content-Type=text/

\ [16.4k/16.4k files][ 40.1 GiB/ 40.1 GiB] 100% Done 143.9 MiB/s ETA 00:00:00   
Operation completed over 16.4k objects/40.1 GiB.                                 


In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:17:37.002848


Previous runs:  
0:42:01.416597  
0:42:14.565295  
0:31:40.728932  
0:23:35.132388

